# Spark SQL y vistas temporales

### Preparación de entorno y análisis de dataset

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.window import Window

# Crear la sesión de Spark
spark = SparkSession.builder.appName("PySpark03").getOrCreate()

In [ ]:
# Cargar datos
df = spark.read.option("header", True).csv("../../data/taxi_zone_lookup.csv")

# Ver las primeras filas
df.show(5)

# Esquema del DataFrame
df.printSchema()

### Creación de una tabla temporal

In [ ]:
# Crear vista temporal
df.createOrReplaceTempView("taxi_zones")

### Consultas SQL

PySpark te permite realizar consultas SQL directamente sobre las tablas creadas. Por ejemplo, si tratáramos de responder a la pregunta "¿Cuántas zonas hay por borough?", podríamos formular la siguiente query:
```sql SELECT Borough, COUNT(DISTINCT Zone) AS num_zones
FROM taxi_zones
GROUP BY Borough
ORDER BY num_zones DESC 

Con Spark, podemos lanzarla directamente como una query de  SQL sin tener que traducirla a PySpark.

In [ ]:
spark.sql("""
    SELECT Borough, COUNT(DISTINCT Zone) AS num_zones
    FROM taxi_zones
    GROUP BY Borough
    ORDER BY num_zones DESC
""").show()

Ejemplo 2: "¿Cuántos LocationID hay por zona de servicio?"

In [ ]:
spark.sql("""
    SELECT service_zone, COUNT(*) AS location_count
    FROM taxi_zones
    GROUP BY service_zone
""").show()

### Función explain()

In [ ]:
spark.sql("""
    SELECT service_zone, COUNT(*) AS location_count
    FROM taxi_zones
    GROUP BY service_zone
""").explain(True)

### Join con Spark SQL

In [ ]:
# Cargar el dataset de viajes
trips_df = spark.read.parquet("../../data/yellow_tripdata_2023-01.parquet")

# Mostrar algunas filas
trips_df.show(5)

# Ver el esquema
trips_df.printSchema()

¿Cuáles son las zonas con más viajes iniciados? 

In [10]:
trips_df.createOrReplaceTempView("yellow_trips")

In [ ]:
spark.sql("""
    SELECT z.Zone AS pickup_zone, COUNT(*) AS num_trips
    FROM yellow_trips t
    JOIN taxi_zones z
    ON t.PULocationID = z.LocationID
    GROUP BY z.Zone
    ORDER BY num_trips DESC
    LIMIT 10
""").show()

In [12]:
spark.stop()